<a href="https://colab.research.google.com/github/alinemsm/kaggle-shuffle-text/blob/main/cooccurrence_santa_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -U git+https://github.com/amitness/colab-connect.git


  Cloning https://github.com/amitness/colab-connect.git to /tmp/pip-req-build-dds0dwkc
  Running command git clone --filter=blob:none --quiet https://github.com/amitness/colab-connect.git /tmp/pip-req-build-dds0dwkc
  Resolved https://github.com/amitness/colab-connect.git to commit 2cb011c5a4a04c13da0a08f8a0e018fe7bf0b09e
  Preparing metadata (setup.py) ... done
  Created wheel for colabconnect: filename=colabconnect-0.0.8-py3-none-any.whl size=4546 sha256=e28f2e1bf3c888a2a9cf9323b154ad2aaa86ab65350480fa173229e72c6a947b
  Stored in directory: /tmp/pip-ephem-wheel-cache-j37ocw6f/wheels/f5/6a/d6/b08b1226dbea3b0c022c25db8a99a9b6cad2973d7a8894c663
Successfully built colabconnect


In [ ]:
from colabconnect import colabconnect
colabconnect()


Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Installing python libraries...
Ran: pip3 install --user flake8 black ipywidgets twine
Ran: pip3 install -U ipykernel
Ran: apt install htop -y
Installing vscode-cli...
Ran: curl -Lk https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64 --output vscode_cli.tar.gz
Ran: tar -xf vscode_cli.tar.gz
Starting the tunnel
Starting the tunnel
- Ready!
- Open VSCode on your laptop and open the command prompt
- Select: 'Remote-Tunnels: Connect to Tunnel' to connect to colab
Logs:

[2025-01-24 15:27:48] info [tunnels::connections::relay_tunnel_host] Opened new client on channel 2
[2025-01-24 15:27:48] info [russh::server] wrote id
[2025-01-24 15:27:48] info [russh::server] read other id
[2025-01-24 15:27:48] info [russh::server] session is running
[2025-01-24 15:27:50] info [rpc.0] Checking /root/.vscode/cli/servers/Stable-4bc113

KeyboardInterrupt: 

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Optional, Union
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
from collections import Counter, deque
from tqdm import tqdm
from datetime import datetime
import time
import gc
from contextlib import contextmanager
import json
import logging
from dataclasses import dataclass
from torch.amp import autocast
import math

@dataclass
class CooccurrenceData:
    """Store and manage co-occurrence statistics."""
    bigram_scores: Dict[Tuple[str, str], float]
    trigram_scores: Dict[Tuple[str, str, str], float]

    @classmethod
    def from_files(cls, bigram_file: str, trigram_file: str):
        """Load co-occurrence data from JSON files."""
        with open(bigram_file, 'r') as f:
            bigram_data = json.load(f)
        with open(trigram_file, 'r') as f:
            trigram_data = json.load(f)

        # Convert string keys to tuples
        bigram_scores = {tuple(k.split('|')): v for k, v in bigram_data.items()}
        trigram_scores = {tuple(k.split('|')): v for k, v in trigram_data.items()}

        return cls(bigram_scores, trigram_scores)

    def get_bigram_score(self, w1: str, w2: str) -> float:
        """Get bigram co-occurrence score."""
        return self.bigram_scores.get((w1, w2), 0.0)

    def get_trigram_score(self, w1: str, w2: str, w3: str) -> float:
        """Get trigram co-occurrence score."""
        return self.trigram_scores.get((w1, w2, w3), 0.0)

    def sequence_score(self, words: List[str]) -> float:
        """Calculate overall sequence score based on co-occurrences."""
        score = 0.0
        # Bigram scores
        for i in range(len(words)-1):
            score += self.get_bigram_score(words[i], words[i+1])

        # Trigram scores
        for i in range(len(words)-2):
            score += self.get_trigram_score(words[i], words[i+1], words[i+2])

        return score / (len(words) - 1)  # Normalize by sequence length


class SmartSequenceOptimizer:
    """Enhanced optimizer with improved error handling and memory management."""

    def __init__(self, model_path: str, checkpoint_dir: str, device: str = 'cuda'):
      self.checkpoint_dir = checkpoint_dir
      self.use_mixed_precision = device == 'cuda'
      self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

      # Initialize tokenizer first
      self._init_tokenizer(model_path)
      # Then model and loss function
      self._init_model(model_path)
      self._init_loss_function()

    def _init_model(self, model_path: str):
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16,
                low_cpu_mem_usage=True,
                device_map='auto',
                max_memory={0: "12GiB"},
                offload_folder="offload"  # Add offload folder for meta device
            )
            self.model.eval()
        except Exception as e:
            logger.error(f"Failed to load model: {str(e)}")
            raise

    def _init_tokenizer(self, model_path: str):
      self.tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
      if self.tokenizer.pad_token is None:
          self.tokenizer.pad_token = self.tokenizer.eos_token
          self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def _init_loss_function(self):
        """Initialize the loss function."""
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none').to(self.device)

    @torch.no_grad()
    def calculate_perplexity(self, text: str, max_retries: int = 3) -> float:
      for attempt in range(max_retries):
          try:
              with autocast('cuda', enabled=self.use_mixed_precision):
                  return self._calculate_perplexity_inner(text)
          except Exception as e:
              logger.warning(f"Perplexity calculation failed (attempt {attempt + 1}): {str(e)}")
              if attempt == max_retries - 1:
                  logger.error(f"All perplexity calculation attempts failed for text: {text}")
                  return float('inf')
              time.sleep(1)

    def verify_permutation(self, original: str, permuted: str) -> bool:
      """Verify if permuted text is valid permutation of original."""
      original_words = Counter(original.split())
      permuted_words = Counter(permuted.split())
      return original_words == permuted_words

    def _calculate_perplexity_inner(self, text: str) -> float:
        """Inner function for perplexity calculation."""
        # Add special tokens
        text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"

        # Tokenize
        inputs = self.tokenizer(
            text_with_special,
            return_tensors='pt',
            add_special_tokens=False,
            max_length=1024,
            truncation=True,
            padding=True
        )

        # Remove token_type_ids if present
        if 'token_type_ids' in inputs:
            inputs.pop('token_type_ids')

        # Move to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Get model outputs
        outputs = self.model(**inputs, use_cache=True)
        logits = outputs.logits

        # Calculate loss
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = inputs['input_ids'][..., 1:].contiguous()

        loss = self.loss_fct(
            shift_logits.view(-1, shift_logits.size(-1)),
            shift_labels.view(-1)
        )

        # Normalize by sequence length
        seq_length = inputs['attention_mask'].sum()
        avg_loss = loss.sum() / seq_length

        # Handle invalid values
        if torch.isnan(avg_loss) or torch.isinf(avg_loss):
            return float('inf')

        # Calculate final perplexity with clamping
        perplexity = torch.exp(torch.clamp(avg_loss, 0, 20)).item()
        return min(perplexity, 1e6)


class EnhancedSequenceOptimizer(SmartSequenceOptimizer):
    """Enhanced optimizer that uses co-occurrence data."""

    def __init__(self, model_path: str, checkpoint_dir: str,
                 cooccurrence_data: CooccurrenceData,
                 device: str = 'cuda'):
        super().__init__(model_path, checkpoint_dir, device)
        self.cooccurrence_data = cooccurrence_data

    def calculate_combined_score(self, text: str, perplexity: float,
                               cooccurrence_weight: float = 0.3) -> float:
        """Combine perplexity and co-occurrence scores."""
        words = text.split()
        cooccurrence_score = self.cooccurrence_data.sequence_score(words)

        # Normalize scores (lower is better for both)
        return (1 - cooccurrence_weight) * perplexity - cooccurrence_weight * cooccurrence_score

    def get_smart_neighbor(self, current_sequence: List[str]) -> Tuple[List[str], str]:
        """Generate neighbor states using co-occurrence data."""
        move_type = random.choice([
            'smart_swap', 'smart_segment_swap',
            'cooccurrence_guided_shuffle', 'random_move'
        ])

        neighbor = current_sequence.copy()

        if move_type == 'smart_swap':
            # Find pairs of words that might improve co-occurrence scores
            best_swap = (0, 1)
            best_improvement = float('-inf')

            # Sample some random pairs to avoid checking all possibilities
            pairs = random.sample([(i, j) for i in range(len(neighbor))
                                 for j in range(i+1, len(neighbor))],
                                min(20, len(neighbor) * (len(neighbor)-1) // 2))

            for i, j in pairs:
                # Calculate score difference if we swap
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                new_score = self.cooccurrence_data.sequence_score(neighbor)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]  # Swap back

                improvement = new_score - self.cooccurrence_data.sequence_score(current_sequence)
                if improvement > best_improvement:
                    best_improvement = improvement
                    best_swap = (i, j)

            # Perform best swap found
            i, j = best_swap
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

        elif move_type == 'smart_segment_swap':
            # Find good segments to swap based on trigram scores
            seg_len = random.randint(2, min(4, len(neighbor)-1))
            start = random.randint(0, len(neighbor)-seg_len)

            # Look for another segment with good trigram transition
            best_target = start
            best_score = float('-inf')

            for target in range(len(neighbor)-seg_len):
                if abs(target - start) < seg_len:
                    continue

                # Try swapping segments and check score
                temp = neighbor.copy()
                temp[start:start+seg_len], temp[target:target+seg_len] = (
                    temp[target:target+seg_len], temp[start:start+seg_len]
                )
                score = self.cooccurrence_data.sequence_score(temp)

                if score > best_score:
                    best_score = score
                    best_target = target

            # Perform best segment swap
            neighbor[start:start+seg_len], neighbor[best_target:best_target+seg_len] = (
                neighbor[best_target:best_target+seg_len], neighbor[start:start+seg_len]
            )

        elif move_type == 'cooccurrence_guided_shuffle':
            # Shuffle a segment while trying to maintain good co-occurrences
            seg_len = random.randint(3, min(6, len(neighbor)))
            start = random.randint(0, len(neighbor)-seg_len)
            segment = neighbor[start:start+seg_len]

            # Try multiple shuffles and pick the best
            best_segment = segment
            best_score = float('-inf')

            for _ in range(5):
                random.shuffle(segment)
                temp = neighbor.copy()
                temp[start:start+seg_len] = segment
                score = self.cooccurrence_data.sequence_score(temp)

                if score > best_score:
                    best_score = score
                    best_segment = segment.copy()

            neighbor[start:start+seg_len] = best_segment

        else:  # random_move - maintain some randomness
            i, j = random.sample(range(len(neighbor)), 2)
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

        return neighbor, move_type

    def optimize_sequence(
        self,
        text: str,
        max_iterations: int = 2000,
        n_chains: int = 5,
        base_temperature: float = 10.0,
        cooling_rate: float = 0.999,
        cooling_type: str = 'exponential',
        min_temp: float = 0.1,
        checkpoint_frequency: int = 100,
        cooccurrence_weight: float = 0.3
    ) -> Tuple[str, float]:
        """Enhanced optimization using co-occurrence data."""
        words = text.split()
        chains = []

        # Initialize chains with combined scoring
        initial_perplexity = self.calculate_perplexity(" ".join(words))
        initial_score = self.calculate_combined_score(text, initial_perplexity, cooccurrence_weight)

        for i in range(n_chains):
            chains.append({
                'sequence': list(words),
                'score': initial_score,
                'temperature': base_temperature * (1.0 - 0.2 * i),
                'steps_without_improvement': 0,
                'best_sequence': list(words),
                'best_score': float('inf')
            })

        global_best_sequence = list(words)
        global_best_score = initial_score

        pbar = tqdm(total=max_iterations * n_chains)

        try:
            for iteration in range(max_iterations):
                for chain_idx, chain in enumerate(chains):
                    # Generate neighbor using co-occurrence data
                    neighbor, move_type = self.get_smart_neighbor(chain['sequence'])
                    neighbor_text = " ".join(neighbor)

                    # Calculate combined score
                    perplexity = self.calculate_perplexity(neighbor_text)
                    neighbor_score = self.calculate_combined_score(
                        neighbor_text, perplexity, cooccurrence_weight
                    )

                    # Accept/reject based on combined score
                    delta = chain['score'] - neighbor_score
                    accept_prob = np.exp(delta / (chain['temperature'] + 1e-6))

                    if neighbor_score < chain['score'] or random.random() < accept_prob:
                        chain['sequence'] = neighbor
                        chain['score'] = neighbor_score

                        if neighbor_score < chain['best_score']:
                            chain['best_score'] = neighbor_score
                            chain['best_sequence'] = list(neighbor)
                            chain['steps_without_improvement'] = 0

                            if neighbor_score < global_best_score:
                                global_best_score = neighbor_score
                                global_best_sequence = list(neighbor)
                                pbar.set_description(f"Best score: {global_best_score:.2f}")

                    # Update temperature
                    if cooling_type == 'exponential':
                        chain['temperature'] *= cooling_rate
                    elif cooling_type == 'linear':
                        chain['temperature'] = max(
                            min_temp,
                            base_temperature * (1 - iteration/max_iterations)
                        )

                    pbar.update(1)

                # Early stopping checks
                if all(chain['temperature'] < min_temp for chain in chains):
                    break

        finally:
            pbar.close()

        return " ".join(global_best_sequence), global_best_score

def optimize_with_cooccurrence(
    previous_results_file: str,
    bigram_file: str,
    trigram_file: str,
    output_dir: str,
    model_path: str,
    checkpoint_dir: str
):
    """Main optimization function using co-occurrence data."""
    # Load co-occurrence data
    cooccurrence_data = CooccurrenceData.from_files(bigram_file, trigram_file)

    # Initialize optimizer
    optimizer = EnhancedSequenceOptimizer(
        model_path=model_path,
        checkpoint_dir=checkpoint_dir,
        cooccurrence_data=cooccurrence_data,
        device='cuda'
    )

    # Load previous results
    df = pd.read_csv(previous_results_file)

    # Process each text
    results = []
    for idx, row in df.iterrows():
        text = row['text']
        optimized_text, score = optimizer.optimize_sequence(
            text,
            max_iterations=2000,
            n_chains=5,
            cooccurrence_weight=0.3  # Adjust based on effectiveness
        )
        results.append({
            'id': row['id'],
            'original_text': text,
            'optimized_text': optimized_text,
            'score': score
        })

        # Save intermediate results
        if (idx + 1) % 5 == 0:
            interim_df = pd.DataFrame(results)
            interim_df.to_csv(f"{output_dir}/interim_results_{idx+1}.csv", index=False)

    # Save final results
    final_df = pd.DataFrame(results)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    final_df.to_csv(f"{output_dir}/final_results_{timestamp}.csv", index=False)

    return final_df

In [ ]:
def setup_paths(
    previous_results_file: str,
    bigram_file: str = 'cooccurrence.json',
    trigram_file: str = 'trigram_cooccurrence.json'
) -> Tuple[str, str, str, str, str, str]:
    """Setup paths for model, data, and co-occurrence files."""
    # Download competition data and model
    model_path = kagglehub.model_download('google/gemma-2/Transformers/gemma-2-9b/2')

    # Set up Google Drive base path
    drive_base = '/content/drive/MyDrive/santa_2024_results'
    os.makedirs(drive_base, exist_ok=True)

    # Define paths for all required files
    previous_results_path = os.path.join(drive_base, previous_results_file)
    bigram_path = os.path.join(drive_base, bigram_file)
    trigram_path = os.path.join(drive_base, trigram_file)

    # Create directories for output and checkpoints
    output_dir = os.path.join(drive_base, 'enhanced_results')
    checkpoint_dir = os.path.join(drive_base, 'enhanced_checkpoints')
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Verify all required files exist
    required_files = {
        'Previous results': previous_results_path,
        'Bigram data': bigram_path,
        'Trigram data': trigram_path
    }

    for name, path in required_files.items():
        if not os.path.exists(path):
            raise ValueError(f"{name} file not found: {path}")

    print(f"Model path: {model_path}")
    print(f"Previous results path: {previous_results_path}")
    print(f"Bigram data path: {bigram_path}")
    print(f"Trigram data path: {trigram_path}")
    print(f"Output directory: {output_dir}")
    print(f"Checkpoint directory: {checkpoint_dir}")

    return model_path, previous_results_path, bigram_path, trigram_path, output_dir, checkpoint_dir

def run_optimization(
    previous_results_file: str,
    text_indices: Optional[List[int]] = None,  # List of indices to optimize, None for all
    start_from: int = 0  # Resume from this index
):
    """
    Main function to run the optimization process.

    Args:
        previous_results_file: Name of the CSV file with texts to optimize
        text_indices: Optional list of text indices to optimize. If None, optimizes all texts
        start_from: Index to resume optimization from (useful for continuing interrupted runs)
    """
    # Mount Google Drive if not already mounted
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # Setup all paths
    model_path, previous_results_path, bigram_path, trigram_path, output_dir, checkpoint_dir = setup_paths(
        previous_results_file=previous_results_file
    )

    # Load co-occurrence data
    cooccurrence_data = CooccurrenceData.from_files(bigram_path, trigram_path)

    # Initialize optimizer with co-occurrence data
    optimizer = EnhancedSequenceOptimizer(
        model_path=model_path,
        checkpoint_dir=checkpoint_dir,
        cooccurrence_data=cooccurrence_data,
        device='cuda'
    )

    # Load previous results
    df = pd.read_csv(previous_results_path)
    print(f"Loaded {len(df)} texts from previous results")

    # Determine which texts to optimize
    if text_indices is None:
        text_indices = list(range(len(df)))
    else:
        # Validate indices
        invalid_indices = [idx for idx in text_indices if idx >= len(df)]
        if invalid_indices:
            raise ValueError(f"Invalid text indices: {invalid_indices}")
        print(f"Will optimize {len(text_indices)} selected texts")

    # Create results directory with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_dir = os.path.join(output_dir, f'optimization_run_{timestamp}')
    os.makedirs(results_dir, exist_ok=True)

    # Initialize results tracking
    all_results = []

    # Load existing results if continuing from previous run
    if start_from > 0:
        last_results_file = os.path.join(results_dir, 'latest_results.csv')
        if os.path.exists(last_results_file):
            existing_results = pd.read_csv(last_results_file)
            all_results.extend(existing_results.to_dict('records'))
            print(f"Loaded {len(all_results)} existing results")

    # Process selected texts
    for idx in tqdm(text_indices[start_from:], desc="Processing texts"):
        row = df.iloc[idx]
        text = row['text']
        print(f"\nProcessing text {idx}/{len(df)-1} (ID: {row['id']})...")

        try:
            optimized_text, score = optimizer.optimize_sequence(
                text,
                max_iterations=2000,
                n_chains=5,
                base_temperature=5.0,
                cooling_rate=0.997,
                cooccurrence_weight=0.3
            )

            # Verify permutation
            if not optimizer.verify_permutation(text, optimized_text):
                print(f"Warning: Invalid permutation found for text {idx}")
                continue

            result = {
                'id': row['id'],
                'index': idx,
                'original_text': text,
                'optimized_text': optimized_text,
                'score': score,
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            all_results.append(result)

            # Save after each text is processed
            results_df = pd.DataFrame(all_results)

            # Save latest results (overwrite)
            latest_file = os.path.join(results_dir, 'latest_results.csv')
            results_df.to_csv(latest_file, index=False)

            # Save individual result
            individual_file = os.path.join(results_dir, f'text_{idx}_result.csv')
            pd.DataFrame([result]).to_csv(individual_file, index=False)

            print(f"Saved results for text {idx} (Score: {score:.2f})")

        except Exception as e:
            print(f"Error processing text {idx}: {str(e)}")
            # Save error log
            error_log = os.path.join(results_dir, 'error_log.txt')
            with open(error_log, 'a') as f:
                f.write(f"Error processing text {idx} at {datetime.now()}: {str(e)}\n")

    # Save final consolidated results
    final_df = pd.DataFrame(all_results)
    final_file = os.path.join(results_dir, f'final_results.csv')
    final_df.to_csv(final_file, index=False)
    print(f"\nFinal results saved to {final_file}")

    return final_df

def optimize_specific_texts(
    previous_results_file: str,
    text_indices: List[int],
    start_from: int = 0
):
    """
    Helper function to optimize specific texts.

    Args:
        previous_results_file: Name of the CSV file with texts to optimize
        text_indices: List of text indices to optimize
        start_from: Index in text_indices to start from (for resuming)
    """
    results = run_optimization(
        previous_results_file=previous_results_file,
        text_indices=text_indices,
        start_from=start_from
    )
    return results


# Example 1: Optimize specific texts
results = optimize_specific_texts(
    previous_results_file="best_sample_submission.csv",
    text_indices=[2]  # Only optimize texts at these indices
)

# Example 2: Optimize all texts
# results = run_optimization(
#     previous_results_file="best_sample_submission.csv"
# )

# Example 3: Resume optimization from a specific index
# results = run_optimization(
#     previous_results_file="best_sample_submission.csv",
#     start_from=3  # Resume from the 4th text
# )


  0%|          | 0.00/856 [00:00<?, ?B/s]

100%|██████████| 856/856 [00:00<00:00, 189kB/s]


100%|██████████| 21.9k/21.9k [00:00<00:00, 8.30MB/s]



100%|██████████| 1.62k/1.62k [00:00<00:00, 1.08MB/s]



  0%|          | 0.00/4.62G [00:00<?, ?B/s]



  0%|          | 0.00/4.62G [00:00<?, ?B/s]

...

Loaded 6 texts from previous results
Will optimize 1 selected texts


Processing texts:   0%|          | 0/1 [00:00<?, ?it/s]


Processing text 2/5 (ID: 2)...


Streaming output truncated to the last 5000 lines.
Processing texts: 100%|██████████| 1/1 [7:31:55<00:00, 27115.78s/it]

Saved results for text 2 (Score: 207.40)

Final results saved to /content/drive/MyDrive/santa_2024_results/enhanced_results/optimization_run_20250120_090759/final_results.csv
